In [22]:
### Equation of motion for 3D double pendulum ###
import sys
sys.path.insert(0, "..")
import sympy.physics.mechanics as me
import sympy as sp
import gen_matlab_class as gm

t = sp.symbols('t')
q = me.dynamicsymbols('q0A, q1A, q2A, q3A, q0B, q1B, q2B, q3B')  # quaternion
u = me.dynamicsymbols('u1A, u2A, u3A, u1B, u2B, u3B')  # angular velocities
l, m, g, Ixx, Iyy, Izz, c = sp.symbols('l m g Ixx Iyy Izz c')  # 

N = me.ReferenceFrame('frame_ground')
N0 = me.Point('point_ground')
N0.set_vel(N,0)

A = me.ReferenceFrame('A')
B = me.ReferenceFrame('B')
mA = me.Point('mA')
mB = me.Point('mB')
ABj = me.Point('ABj')

# rotate joint by quaternion and apply angular velocities
# first joint is in the world frame center
A.orient(N, 'Quaternion', [q[0], q[1], q[2], q[3]])
A.set_ang_vel(N, u[0]*A.x + u[1]*A.y + u[2]*A.z)

# set masscenter of the first body
mA.set_pos(N0, -l/2 * A.z)
mA.v2pt_theory(N0,N,A)
FG1 = [(mA, -m * g * N.z)]

# set the position of the joint between the two bodies
ABj.set_pos(N0, -l * A.z)
ABj.v2pt_theory(N0,N,A)

# orient second body from the first body
B.orient(A, 'Quaternion', [q[4], q[5], q[6], q[7]])
B.set_ang_vel(A, u[3]*B.x + u[4]*B.y + u[5]*B.z)
mB.set_pos(ABj, -l/2 * B.z)
mB.v2pt_theory(ABj,N,B)
I1 = me.inertia(A, Ixx, Iyy, Izz)
I2 = me.inertia(B, Ixx, Iyy, Izz)

BODY = []
BODY.append(me.RigidBody('Abody', mA, A, m, (I1, mA)))
BODY.append(me.RigidBody('Bbody', mB, B, m, (I2, mB)))

FG2 = [(mB, -m * g * N.z)]
DAMP = []
DAMP.append((A,-c*(u[0]*A.x + u[1]*A.y + u[2]*A.z)))
DAMP.append((B,-c*(u[3]*B.x + u[4]*B.y + u[5]*B.z)))
DAMP.append((A, c*(u[3]*B.x + u[4]*B.y + u[5]*B.z)))

kindeq = []

# set kinematic differential equations - see Quaternions and Dynamics, page 9, equation 18
for i in range(2):
    kindeq.append(q[0+i*4].diff(t) - 0.5 * (-u[0+i*3]*q[1+i*4] - u[1+i*3]*q[2+i*4] - u[2+i*3]*q[3+i*4]))
    kindeq.append(q[1+i*4].diff(t) - 0.5 * (u[0+i*3]*q[0+i*4] + u[2+i*3]*q[2+i*4] - u[1+i*3]*q[3+i*4]))
    kindeq.append(q[2+i*4].diff(t) - 0.5 * (u[1+i*3]*q[0+i*4] - u[2+i*3]*q[1+i*4] + u[0+i*3]*q[3+i*4]))
    kindeq.append(q[3+i*4].diff(t) - 0.5 * (u[2+i*3]*q[0+i*4] + u[1+i*3]*q[1+i*4] - u[0+i*3]*q[2+i*4]))
F = FG1+FG2
kindeq
KM = me.KanesMethod(N, q_ind=q, u_ind=u, kd_eqs=kindeq)
(fr, frstar) = KM.kanes_equations(BODY, (FG1+FG2+DAMP))

In [23]:
# substituting dynamicsymbols for symbolics so it can be printed into matlab
qm = sp.symbols('qm0:8')
um = sp.symbols('um1:7')
subs_q = {q[i]: qm[i] for i in range(len(q))}
subs_u = {u[i]: um[i] for i in range(len(u))}
mm = me.msubs(KM.mass_matrix_full,subs_q,subs_u)
fo = me.msubs(KM.forcing_full,subs_q,subs_u)

from importlib import reload
reload(gm)
parameters_dict = {l:'l',m:'m',g:'g',Ixx:'Ixx',Iyy:'Iyy',Izz:'Izz',c:'c'}
gm.MatlabFunction(function = fo,fun_name = 'fo_python',assignto = 'fo_python',coordinates = qm,speeds = um,inputs = {},parameters = parameters_dict)
gm.MatlabFunction(function = mm,fun_name = 'mm_python',assignto = 'mm_python',coordinates = qm,speeds = um,inputs = {},parameters = parameters_dict)

In [13]:
# Notes
# this is how the vector of coordinates looks like
print('q vector:',KM.q)
# this is how the vector of speeds looks like
print('u vector:',KM.u)
# ODE is in the form: M*x_dot = fe, where x_dot = [q,u]_dot

q vector: Matrix([[q0A(t)], [q1A(t)], [q2A(t)], [q3A(t)], [q0B(t)], [q1B(t)], [q2B(t)], [q3B(t)]])
u vector: Matrix([[u1A(t)], [u2A(t)], [u3A(t)], [u1B(t)], [u2B(t)], [u3B(t)]])


In [13]:
### BY JASON MOORE ###
# just different way how create equations of motion,
# here one more speed is introduced, but this approach has a singularity when q0=0

import sympy.physics.mechanics as me
import sympy as sp

t = sp.symbols('t')
q0A, q1A, q2A, q3A, q0B, q1B, q2B, q3B = me.dynamicsymbols('q0A, q1A, q2A, q3A, q0B, q1B, q2B, q3B')  # quaternion
w1A, w2A, w3A, w1B, w2B, w3B = me.dynamicsymbols('w1A, w2A, w3A, w1B, w2B, w3B')  # angular velocities
u0A, u0B = me.dynamicsymbols('u0A u0B')
l, m, g, Ixx, Iyy, Izz = sp.symbols('l m g Ixx Iyy Izz')  # 

N = me.ReferenceFrame('frame_ground')
N0 = me.Point('point_ground')
N0.set_vel(N,0)

A = me.ReferenceFrame('A')
B = me.ReferenceFrame('B')
mA = me.Point('mA')
mB = me.Point('mB')
ABj = me.Point('ABj')


A.orient(N, 'Quaternion', [q0A, q1A, q2A, q3A])

N_w_A = A.ang_vel_in(N)
 
kinematical1 = sp.Matrix([
    u0A - q0A.diff(t),
    w1A - N_w_A.dot(A.x),
    w2A - N_w_A.dot(A.y),
    w3A - N_w_A.dot(A.z),
])

B.orient(A, 'Quaternion', [q0B, q1B, q2B, q3B])

A_w_B = B.ang_vel_in(A)

kinematical2 = (sp.Matrix([
    u0B - q0B.diff(t),
    w1B - A_w_B.dot(B.x),
    w2B - A_w_B.dot(B.y),
    w3B - A_w_B.dot(B.z),
]))

A.set_ang_vel(N, w1A*A.x + w2A*A.y + w3A*A.z)
B.set_ang_vel(A, w1B*B.x + w2B*B.y + w3B*B.z)

mA.set_pos(N0, -l/2 * A.z)
mA.v2pt_theory(N0,N,A)
FG1 = [(mA, -m * g * N.z)]

ABj.set_pos(N0, -l * A.z)
ABj.v2pt_theory(N0,N,A)


mB.set_pos(ABj, -l/2 * B.z)
mB.v2pt_theory(ABj,N,B)


I1 = me.inertia(A, Ixx, Iyy, Izz)
I2 = me.inertia(B, Ixx, Iyy, Izz)

BODY = []
BODY.append(me.RigidBody('Abody', mA, A, m, (I1, mA)))
BODY.append(me.RigidBody('Bbody', mB, B, m, (I2, mB)))

kinematical = sp.Matrix([[kinematical1],[kinematical2]])

FG2 = [(mB, -m * g * N.z)]

F = FG1+FG2
holonomic = sp.Matrix([[q0A**2 + q1A**2 + q2A**2 + q3A**2 - 1],
                       [q0B**2 + q1B**2 + q2B**2 + q3B**2 - 1]])
kane = me.KanesMethod(
    N,
    [q1A, q2A, q3A, q1B, q2B, q3B],
    [w1A, w2A, w3A, w1B, w2B, w3B],
    kd_eqs=kinematical,
    q_dependent=[q0A,q0B],
    u_dependent=[u0A,u0B],
    configuration_constraints=holonomic,
    velocity_constraints=holonomic.diff(t),
)
(fr, frstar) = kane.kanes_equations(BODY, (FG1+FG2))

In [76]:
sp.shape(kane.mass_matrix_full)

(16, 16)

In [74]:
qm = sp.symbols('q1U q2U q3U q1L q2L q3L q0U q0L')
um = sp.symbols('w1U w2U w3U w1L w2L w3L u0U u0L')
kane_q = list(kane.q)
kane_u = list(kane.u)
subs_q = {kane_q[i]: qm[i] for i in range(len(qm))}
subs_u = {kane_u[i]: um[i] for i in range(len(um))}
mm = me.msubs(kane.mass_matrix_full,subs_q,subs_u)
fe = me.msubs(kane.forcing_full,subs_q,subs_u)
# print(sp.octave_code(mm,assign_to = 'mm'))
# print(sp.octave_code(fe,assign_to = 'fe'))

In [23]:
import sympy as sp
a,b,c,d = sp.symbols('a b c d')

G = sp.Matrix([[-b, a, d, -c],
                [-c,-d, a, b],
                [-d, c, -b, a]])
T = sp.Matrix([[-b/a, -c/a, -d/a],[sp.eye(3)]])
(2*G*T).inv()

Matrix([
[ a/(2*a**2 + 2*b**2 + 2*c**2 + 2*d**2), -d/(2*a**2 + 2*b**2 + 2*c**2 + 2*d**2),  c/(2*a**2 + 2*b**2 + 2*c**2 + 2*d**2)],
[ d/(2*a**2 + 2*b**2 + 2*c**2 + 2*d**2),  a/(2*a**2 + 2*b**2 + 2*c**2 + 2*d**2), -b/(2*a**2 + 2*b**2 + 2*c**2 + 2*d**2)],
[-c/(2*a**2 + 2*b**2 + 2*c**2 + 2*d**2),  b/(2*a**2 + 2*b**2 + 2*c**2 + 2*d**2),  a/(2*a**2 + 2*b**2 + 2*c**2 + 2*d**2)]])